In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import ScalarFormatter
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

<h4>Данные</h4>

In [3]:
country_names = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_COUNTRY.csv")

In [52]:
country_names

,COUNTRY_ID,COUNTRY_NAME_EN
0,AFG,Afghanistan
1,ALB,Albania
2,DZA,Algeria
3,ASM,American Samoa
4,AND,Andorra
...,...,...
236,WLF,Wallis and Futuna Islands
237,WSM,Samoa
238,YEM,Yemen
239,COD,Democratic Republic of the Congo


In [4]:
labels = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_students_label.csv")
# labels = labels['INDICATOR_ID'].astype(int)
labels.columns

Index(['INDICATOR_ID', 'INDICATOR_LABEL_EN'], dtype='object')

In [5]:
labels.dtypes

INDICATOR_ID           int64
INDICATOR_LABEL_EN    object
dtype: object

In [6]:
national = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_DATA_NATIONAL.csv")
national.columns

C:\Users\yuliy\AppData\Local\Temp\ipykernel_17988\3021277289.py:1: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  national = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_DATA_NATIONAL.csv")


Index(['indicator_id', 'country_id', 'year', 'value', 'magnitude',
       'qualifier'],
      dtype='object')

In [13]:
#данные въезда в США за 2022 г
usa_add = pd.read_csv(r"C:\Users\yuliy\Education_Projects\students_mobility\usa_inbound.csv")
usa_add['to'] = 'United States'
usa_add = usa_add.rename(columns={'unesco':'from'})
usa_add = usa_add[['from','to','2022']]
usa_add

,from,to,2022
0,Afghanistan,United States,682
1,Albania,United States,"1,087"
2,Algeria,United States,302
3,Andorra,United States,20
4,Angola,United States,553
...,...,...,...
219,Wallis and Futuna Islands,United States,0
220,Western Sahara,United States,-
221,Yemen,United States,305
222,Zambia,United States,555


In [15]:
usa_add_long = pd.melt(usa_add,id_vars=['from','to'],value_vars = ['2022'],var_name='year',value_name='value')
usa_add_long

,from,to,year,value
0,Afghanistan,United States,2022,682
1,Albania,United States,2022,"1,087"
2,Algeria,United States,2022,302
3,Andorra,United States,2022,20
4,Angola,United States,2022,553
...,...,...,...,...
219,Wallis and Futuna Islands,United States,2022,0
220,Western Sahara,United States,2022,-
221,Yemen,United States,2022,305
222,Zambia,United States,2022,555


In [19]:
#данные въезда в Казахстан за 2022 г
kaz_add_long = pd.read_csv(r"C:\Users\yuliy\Education_Projects\students_mobility\kazakhstan_inbound.csv")
kaz_add_long.head()

,from,to,year,value
0,Uzbekistan,Kazakhstan,2022,9571.0
1,Tajikistan,Kazakhstan,2022,361.0
2,Turkmenistan,Kazakhstan,2022,1373.0
3,Azerbaijan,Kazakhstan,2022,116.0
4,Kyrgyzstan,Kazakhstan,2022,597.0


inbound отчеты - обработка и добавление

In [21]:
from_to_labels = labels[~labels['INDICATOR_ID'].isin([26637,26638,43188])]
from_to_labels['from'] = from_to_labels['INDICATOR_LABEL_EN'].apply(lambda x:x.split('from ')[1].split(', both sexes (number)')[0].replace('the ',''))
from_to_labels.head()

C:\Users\yuliy\AppData\Local\Temp\ipykernel_17988\106753272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_to_labels['from'] = from_to_labels['INDICATOR_LABEL_EN'].apply(lambda x:x.split('from ')[1].split(', both sexes (number)')[0].replace('the ',''))


,INDICATOR_ID,INDICATOR_LABEL_EN,from
0,26421,"Africa: Students from Algeria, both sexes (num...",Algeria
1,26422,"Africa: Students from Angola, both sexes (number)",Angola
2,26423,"Africa: Students from Benin, both sexes (number)",Benin
3,26424,"Africa: Students from Botswana, both sexes (nu...",Botswana
4,26425,"Africa: Students from Burkina Faso, both sexes...",Burkina Faso


In [22]:
total_from_to = national[national['indicator_id'].isin(from_to_labels['INDICATOR_ID'].unique())]
total_from_to = pd.merge(total_from_to,from_to_labels,left_on='indicator_id',right_on='INDICATOR_ID',how='left')
total_from_to = pd.merge(total_from_to,country_names,left_on='country_id',right_on='COUNTRY_ID',how='left')
total_from_to = total_from_to.rename(columns={'COUNTRY_NAME_EN':'to'})
total_from_to =total_from_to[['indicator_id','country_id','year','value','from','to']]
print(total_from_to.shape)
total_from_to.head()

(453280, 6)


,indicator_id,country_id,year,value,from,to
0,26421,ABW,2003,0.0,Algeria,Aruba
1,26421,ABW,2004,0.0,Algeria,Aruba
2,26421,ABW,2006,0.0,Algeria,Aruba
3,26421,ABW,2009,0.0,Algeria,Aruba
4,26421,ABW,2010,0.0,Algeria,Aruba


In [32]:
#удалим строки 2022  в США - пустые
total_from_to = total_from_to[~(total_from_to['to']=='United States') & ~(total_from_to['year']==2022)]
total_from_to.head()

,indicator_id,country_id,year,value,from,to


In [23]:
total_from_to = pd.concat([total_from_to,usa_add_long,kaz_add_long])
total_from_to

,indicator_id,country_id,year,value,from,to
0,26421,ABW,2003,0.0,Algeria,Aruba
1,26421,ABW,2004,0.0,Algeria,Aruba
2,26421,ABW,2006,0.0,Algeria,Aruba
3,26421,ABW,2009,0.0,Algeria,Aruba
4,26421,ABW,2010,0.0,Algeria,Aruba
...,...,...,...,...,...,...
25,NaN,NaN,2021,353.0,Afghanistan,Kazakhstan
26,NaN,NaN,2021,583.0,Jordan,Kazakhstan
27,NaN,NaN,2021,NaN,Republic of Korea,Kazakhstan
28,NaN,NaN,2021,NaN,Belarus,Kazakhstan


<h4>Входящие по странам</h4>

На основании отчетов inbound в разрезе страна-страна

In [29]:
inbound = total_from_to[total_from_to['year'].isin([2017,2022])].pivot(index=['from','to'], columns='year', values='value').reset_index()
inbound = inbound[~inbound['from'].isin(['unknown countries'])]
inbound


year,from,to,2017,2022
0,Afghanistan,Albania,NaN,0.0
1,Afghanistan,Andorra,0.0,NaN
2,Afghanistan,Argentina,15.0,16.0
3,Afghanistan,Armenia,0.0,0.0
4,Afghanistan,Australia,44.0,73.0
...,...,...,...,...
24542,Zimbabwe,United Kingdom of Great Britain and Northern I...,924.0,1548.0
24543,Zimbabwe,United Republic of Tanzania,NaN,16.0
24544,Zimbabwe,United States of America,1365.156982,NaN
24545,Zimbabwe,Uzbekistan,0.0,0.0


На основании отчетов inbound в разрезе весь мир -страна

In [305]:
total_inbound = national[national['indicator_id'].isin([26637])]
total_inbound['from'] = 'World'
total_inbound.head()

C:\Users\yuliy\AppData\Local\Temp\ipykernel_19400\538833217.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,indicator_id,country_id,year,value,magnitude,qualifier,from
739099,26637,ABW,2003,91.0,NaN,NaN,World
739100,26637,ABW,2005,40.0,NaN,NaN,World
739101,26637,ABW,2006,169.0,NaN,NaN,World
739102,26637,ABW,2009,107.0,NaN,NaN,World
739103,26637,ABW,2010,63.0,NaN,NaN,World


In [306]:
diff = total_inbound[total_inbound['year'].isin([2017,2022])].pivot(index=['country_id'], columns='year', values='value').reset_index()

diff=pd.merge(diff,country_names,left_on='country_id',right_on='COUNTRY_ID',how='left')
diff.loc[diff['country_id']=='USA',2022]=1057288
# diff.loc[diff['country_id']=='USA',2021]=948519
diff.loc[diff['country_id']=='NLD',2022]=122287
diff.loc[diff['country_id']=='EGY',2017]=51162
diff.loc[diff['country_id']=='THA',2017] = 43821
diff.loc[diff['country_id']=='MEX',2022]=51659
diff.loc[diff['country_id']=='IRN',2022]=94406
diff.loc[diff['country_id']=='KAZ',2022]=26080
diff.loc[diff['country_id']=='UKR',2022]=50870
diff.loc[diff['COUNTRY_NAME_EN']=='Iran (Islamic Republic of)','COUNTRY_NAME_EN']='Iran'
diff.loc[diff['COUNTRY_NAME_EN']=='United Kingdom of Great Britain and Northern Ireland','COUNTRY_NAME_EN']='Great Britain'
diff.loc[diff['COUNTRY_NAME_EN']=='China, Hong Kong Special Administrative Region','COUNTRY_NAME_EN']='Hong Kong'
diff.loc[diff['COUNTRY_NAME_EN']=='China, Macao Special Administrative Region','COUNTRY_NAME_EN']='Macao'

diff = diff[~(diff[2017].isna()) & ~(diff[2022].isna())]
diff = diff[~(diff[2017]==0) & ~(diff[2022]==0)]
diff['2022/2017,%'] = round(diff[2022]/diff[2017]*100,0).astype(int)-100
diff['2022/2017,%'] = diff['2022/2017,%'].apply(lambda x:'+'+str(x) if x>0 else str(x))
diff = diff[diff[2022]>5000]
def group(x):
    if x<20000:
        group_name = '5-20 тыс.'
    elif 20000<=x<50000:
        group_name = '20-50 тыс.'
    elif 50000<=x<100000:
        group_name = '50-100 тыс.'
    else:
        group_name = 'Больше 100 тыс.'
    return group_name
diff['group'] = diff[2022].apply(lambda x: group(x))
diff.set_index('COUNTRY_NAME_EN', inplace=True)
diff.sort_values(by='2022/2017,%',ascending=False)

,country_id,2017,2022,COUNTRY_ID,"2022/2017,%",group
COUNTRY_NAME_EN,,,,,,
Malaysia,MYS,100765.0,100437.0,MYS,0,Больше 100 тыс.
Australia,AUS,381202.0,382007.0,AUS,0,Больше 100 тыс.
India,IND,46703.0,46659.0,IND,0,20-50 тыс.
Italy,ITA,97563.0,89946.0,ITA,-8,50-100 тыс.
Denmark,DNK,33602.0,31375.0,DNK,-7,20-50 тыс.
...,...,...,...,...,...,...
Georgia,GEO,7994.0,17527.0,GEO,+119,5-20 тыс.
Cameroon,CMR,3941.0,8499.0,CMR,+116,5-20 тыс.
Finland,FIN,24168.0,26774.0,FIN,+11,20-50 тыс.


In [307]:
diff[2022].sum()

np.float64(6579656.9140625)

In [308]:
diff[2017].sum()

np.float64(5077737.578125)

In [153]:
diff[2022].sum()/diff[2017].sum()*100

np.float64(129.57851430542215)

In [154]:
diff.columns

Index(['country_id', 2017, 2022, 'COUNTRY_ID', '2022/2017,%', 'group'], dtype='object')

In [146]:
unique_groups

array(['Больше 100 тыс.', 'До 10 тыс.', '50-100 тыс.', '10-50 тыс.'],
      dtype=object)

In [256]:
n_cols = 2  # Number of columns in subplot grid
# unique_groups  = diff['group'].unique()
unique_groups = ['Больше 100 тыс.', '50-100 тыс.', '20-50 тыс.','5-20 тыс.']
num_plots = len(unique_groups)
num_cols = 2  # Maximum of 2 columns
num_rows = (num_plots + num_cols - 1) // num_cols  # Calculate the number of rows
fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=unique_groups)

for i, group in enumerate(unique_groups):
    filtered_df = diff[diff['group'] == group]
    row_index = i // n_cols + 1
    col_index = i % n_cols + 1
    
    trace1 = go.Bar(x=filtered_df.index,
        y=filtered_df[2017],
        name=f'2017 - Group {group}',
        text=[f"{value/1000:.1f}k" if value < 1000 else f"{int(value/1000)}k" for value in filtered_df[2017]],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=16),
        marker_color='blue',
        marker_line=dict(width=3, color=['red' if country == 'RUS' else 'blue' for country in filtered_df.country_id]) 
        )
    trace2 = go.Bar(x=filtered_df.index,
        y=filtered_df[2022],
        name=f'2022 - Group {group}',
        text=[f"{value/1000:.1f}k\n({filtered_df['2022/2017,%'].values[j]}%)" if value < 1000 else f"{int(value/1000)}k\n({filtered_df['2022/2017,%'].values[j]}%)" for j, value in enumerate(filtered_df[2022])],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=16),
        marker_color='green',
        marker_line=dict(width=3, color=['red' if country == 'RUS' else 'green' for country in filtered_df.country_id]) 
        )
    
    fig.add_trace(trace1, row=row_index, col=col_index)
    fig.add_trace(trace2, row=row_index, col=col_index)
        
fig.update_layout(
title='Изменение кол-ва иностранных студентов по группам стран с 2017 по 2022 г.',
# xaxis_title='Country',
yaxis_title='Число студентов, тыс.',
barmode='group',  # Group bars together
height=1400,
width=2600,
plot_bgcolor='white'
)
for i in range(1, num_rows + 1):
    for j in range(1, num_cols + 1):
        fig.update_xaxes(tickangle=45, row=i, col=j)
        fig.update_yaxes(gridcolor='lightgrey', row=i, col=j)
pio.write_image(fig, "Изменение кол-ва иностранных студентов по группам стран с 2017 по 2022 г.png")
fig.show()

<h4>Исходящие по странам</h4>

OE.5T8.40510 
Total outbound internationally mobile tertiary students studying abroad, all countries, both sexes (UIS estimate) (number)

In [297]:
total_outbound = national[national['indicator_id'].isin(['OE.5T8.40510'])]
total_outbound = pd.merge(total_outbound,country_names,left_on='country_id',right_on='COUNTRY_ID',how='left')
total_outbound.head()

,indicator_id,country_id,year,value,magnitude,qualifier,COUNTRY_ID,COUNTRY_NAME_EN
0,OE.5T8.40510,ABW,1998,171.0,NaN,UIS_EST,ABW,Aruba
1,OE.5T8.40510,ABW,1999,127.0,NaN,UIS_EST,ABW,Aruba
2,OE.5T8.40510,ABW,2000,171.0,NaN,UIS_EST,ABW,Aruba
3,OE.5T8.40510,ABW,2001,152.0,NaN,UIS_EST,ABW,Aruba
4,OE.5T8.40510,ABW,2002,324.0,NaN,UIS_EST,ABW,Aruba


In [304]:
world_outbound = total_outbound.groupby(['year'])['value'].sum().reset_index()
fig = px.bar(world_outbound,x='year',y='value',text='value')
fig.show()
# world_outbound

In [295]:
diff = total_outbound[total_outbound['year'].isin([2017,2022])].pivot(index=['country_id','COUNTRY_NAME_EN'], columns='year', values='value').reset_index()
diff = diff.dropna()
diff['2022/2017,%'] = round(diff[2022]/diff[2017]*100,0).astype(int)-100
diff['2022/2017,%'] = diff['2022/2017,%'].apply(lambda x:'+'+str(x) if x>0 else str(x))
diff = diff[diff[2022]>10000]
def group(x):
    if x<20000:
        group_name = '5-20 тыс.'
    elif 20000<=x<50000:
        group_name = '20-50 тыс.'
    elif 50000<=x<100000:
        group_name = '50-100 тыс.'
    elif 100000<=x<200000:
        group_name = '100-200 тыс.'
    else:
        group_name = 'Больше 200 тыс.'
    return group_name
diff['group'] = diff[2022].apply(lambda x: group(x))
diff.loc[diff['COUNTRY_NAME_EN']=='Iran (Islamic Republic of)','COUNTRY_NAME_EN']='Iran'
diff.loc[diff['COUNTRY_NAME_EN']=='United Kingdom of Great Britain and Northern Ireland','COUNTRY_NAME_EN']='Great Britain'
diff.loc[diff['COUNTRY_NAME_EN']=='China, Hong Kong Special Administrative Region','COUNTRY_NAME_EN']='Hong Kong'
diff.loc[diff['COUNTRY_NAME_EN']=='China, Macao Special Administrative Region','COUNTRY_NAME_EN']='Macao'
diff.set_index('COUNTRY_NAME_EN', inplace=True)
diff.sort_values(by='2022/2017,%',ascending=False)
diff

year,country_id,2017,2022,"2022/2017,%",group
COUNTRY_NAME_EN,,,,,
Afghanistan,AFG,29995.0,34323.0,+14,20-50 тыс.
Angola,AGO,15605.0,10797.0,-31,5-20 тыс.
Albania,ALB,17808.0,14819.0,-17,5-20 тыс.
United Arab Emirates,ARE,11261.0,18600.0,+65,5-20 тыс.
Argentina,ARG,8423.0,11700.0,+39,5-20 тыс.
...,...,...,...,...,...
Uzbekistan,UZB,35033.0,150488.0,+330,100-200 тыс.
Viet Nam,VNM,94643.0,134141.0,+42,100-200 тыс.
Yemen,YEM,25019.0,36031.0,+44,20-50 тыс.


In [296]:
n_cols = 2  # Number of columns in subplot grid
# unique_groups  = diff['group'].unique()
unique_groups = ['Больше 200 тыс.','100-200 тыс.', '50-100 тыс.', '20-50 тыс.','5-20 тыс.']
num_plots = len(unique_groups)
num_cols = 2  # Maximum of 2 columns
num_rows = (num_plots + num_cols - 1) // num_cols  # Calculate the number of rows
fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=unique_groups)
diff = diff.sort_values(by=[2022],ascending=False)
for i, group in enumerate(unique_groups):
    filtered_df = diff[diff['group'] == group]
    row_index = i // n_cols + 1
    col_index = i % n_cols + 1
    
    trace1 = go.Bar(x=filtered_df.index,
        y=filtered_df[2017],
        name=f'2017 - Group {group}',
        text=[f"{value/1000:.1f}k" if value < 1000 else f"{int(value/1000)}k" for value in filtered_df[2017]],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=12),
        marker_color='brown',
        marker_line=dict(width=3, color=['black' if country == 'RUS' else 'brown' for country in filtered_df.country_id]) 
        )
    trace2 = go.Bar(x=filtered_df.index,
        y=filtered_df[2022],
        name=f'2022 - Group {group}',
        text=[f"{value/1000:.1f}k<br>({filtered_df['2022/2017,%'].values[j]}%)" if value < 1000 else f"{int(value/1000)}k<br>({filtered_df['2022/2017,%'].values[j]}%)" for j, value in enumerate(filtered_df[2022])],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=12),
        marker_color='orange',
        marker_line=dict(width=3, color=['black' if country == 'RUS' else 'orange' for country in filtered_df.country_id]) 
        )
    
    fig.add_trace(trace1, row=row_index, col=col_index)
    fig.add_trace(trace2, row=row_index, col=col_index)
        
fig.update_layout(
title='Изменение кол-ва иностранных студентов по группам стран с 2017 по 2022 г.',
# xaxis_title='Country',
yaxis_title='Число студентов, тыс.',
barmode='group',  # Group bars together
height=1600,
width=2600,
plot_bgcolor='white'
)
for i in range(1, num_rows + 1):
    for j in range(1, num_cols + 1):
        fig.update_xaxes(tickangle=45, row=i, col=j)
        fig.update_yaxes(gridcolor='lightgrey', row=i, col=j)
pio.write_image(fig, "Изменение кол-ва иностранных студентов по группам стран с 2017 по 2022 г.png")
fig.show()

<h4>Внутри ЦА и России</h4>

In [ ]:
diff = total_from_to[total_from_to['year'].isin([2017,2022])].pivot(index=['indicator_id','country_id','from','to'], columns='year', values='value').reset_index()
diff.loc[((diff['country_id']=='KAZ') & (diff['from']=='Uzbekistan')),2022]=9571
diff = diff[diff[2022]>20000]
diff

In [294]:
unique_labels = list(set(diff['from']).union(set(diff['to'])))
label_to_index = {label: index for index, label in enumerate(unique_labels)}
diff['source'] = diff['from'].map(label_to_index)
diff['target'] = diff['to'].map(label_to_index)
diff

year,indicator_id,country_id,from,to,2017,2022,source,target
39,26421,FRA,Algeria,France,20491.000000,24315.109375,14,31
3979,26454,FRA,Morocco,France,29733.000000,35909.367188,34,31
4458,26458,GBR,Nigeria,United Kingdom of Great Britain and Northern I...,12642.000000,44076.000000,10,4
9345,26503,GBR,United States,United Kingdom of Great Britain and Northern I...,16178.000000,22031.000000,12,4
9666,26508,ARG,Brazil,Argentina,12789.000000,24571.000000,7,3
11203,26522,TUR,Azerbaijan,Turkey,14878.000000,28616.000000,18,13
11814,26528,AUS,China,Australia,128498.000000,87899.000000,28,5
11830,26528,CAN,China,Canada,66161.000000,73032.000000,28,11
11842,26528,DEU,China,Germany,27765.000000,38486.000000,28,20
11849,26528,FRA,China,France,24788.000000,23260.910156,28,31


In [ ]:



fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_labels,  # Use defined labels here
        color="blue"           # Color for all nodes
    ),
    link=dict(
        source=diff['source'],   # Indices correspond to labels
        target=diff['target'],
        value=diff[2022]       # Use values from the 2022 column
    )
)])

fig.update_layout(
    title_text="Sankey Diagram of Country Flows", 
    font_size=10,
    height=1200,
    width=1200,
    plot_bgcolor='white'
)
fig.show()